### 数据展示

In [4]:
import pandas as pd
import numpy as np
from scipy.interpolate import UnivariateSpline
from sklearn import linear_model
import xgboost as xgb
# from ultis import *

道路通行时间：

In [6]:
df = pd.read_csv('new_gy_contest_traveltime_training_data_second.txt',delimiter=';',dtype={'link_ID':object})
df.head()

,link_ID,date,time_interval,travel_time
0,4377906283422600514,2017-05-06,"[2017-05-06 11:04:00,2017-05-06 11:06:00)",3.0
1,3377906289434510514,2017-05-06,"[2017-05-06 10:42:00,2017-05-06 10:44:00)",1.0
2,3377906285934510514,2017-05-06,"[2017-05-06 11:56:00,2017-05-06 11:58:00)",35.2
3,3377906285934510514,2017-05-06,"[2017-05-06 17:46:00,2017-05-06 17:48:00)",26.2
4,3377906287934510514,2017-05-06,"[2017-05-06 10:52:00,2017-05-06 10:54:00)",10.4


time_interval时间间隔，两分钟为单位

travel_time平均通行时间

道理长宽情况：

In [8]:
link_df = pd.read_csv('gy_contest_link_info.txt',delimiter=';',dtype={'link_ID':object})
link_df.head()

,link_ID,length,width,link_class
0,4377906289869500514,57,3,1
1,4377906284594800514,247,9,1
2,4377906289425800514,194,3,1
3,4377906284525800514,839,3,1
4,4377906284422600514,55,12,1


length长度    width宽度    link_class类别

道路之间连接情况：

In [11]:
link_tops = pd.read_csv('gy_contest_link_top_update.txt',delimiter=',',dtype={'link_ID':object})
link_tops.head()

,link_ID,in_links,out_links
0,4377906289869500514,1,1
1,4377906284594800514,1,1
2,4377906289425800514,1,1
3,4377906284525800514,1,1
4,4377906284422600514,2,1


### 任务：预测未来一个月平均通行结果，每两分钟一次
回归任务

构建时间序列，基于前几天或者前几十天的数据预测

### 数据集筛选与标签转换
数据集中有些数据可能由于异常情况导致不适合建模（堵车，维修等）

In [12]:
df.head()

,link_ID,date,time_interval,travel_time
0,4377906283422600514,2017-05-06,"[2017-05-06 11:04:00,2017-05-06 11:06:00)",3.0
1,3377906289434510514,2017-05-06,"[2017-05-06 10:42:00,2017-05-06 10:44:00)",1.0
2,3377906285934510514,2017-05-06,"[2017-05-06 11:56:00,2017-05-06 11:58:00)",35.2
3,3377906285934510514,2017-05-06,"[2017-05-06 17:46:00,2017-05-06 17:48:00)",26.2
4,3377906287934510514,2017-05-06,"[2017-05-06 10:52:00,2017-05-06 10:54:00)",10.4


In [13]:
#截取开始时间
df['time_interval_begin'] = pd.to_datetime(df['time_interval'].map(lambda x: x[1:20]))
df.head()

,link_ID,date,time_interval,travel_time,time_interval_begin
0,4377906283422600514,2017-05-06,"[2017-05-06 11:04:00,2017-05-06 11:06:00)",3.0,2017-05-06 11:04:00
1,3377906289434510514,2017-05-06,"[2017-05-06 10:42:00,2017-05-06 10:44:00)",1.0,2017-05-06 10:42:00
2,3377906285934510514,2017-05-06,"[2017-05-06 11:56:00,2017-05-06 11:58:00)",35.2,2017-05-06 11:56:00
3,3377906285934510514,2017-05-06,"[2017-05-06 17:46:00,2017-05-06 17:48:00)",26.2,2017-05-06 17:46:00
4,3377906287934510514,2017-05-06,"[2017-05-06 10:52:00,2017-05-06 10:54:00)",10.4,2017-05-06 10:52:00


标签转换
<img src="assets/20201202211044.png" width="100%">
我们希望是右边的图，越是正态分布，越好预测

In [16]:
df = df.drop(['time_interval'],axis=1)
df['travel_time'] = np.log1p(df['travel_time'])

筛选方法

In [17]:
#剔除一些离群点：如travel_time突然有几百分钟的时间，可能是意外、道路维修或者统计错误
def quantile_clip(group):
    # 选择一定的百分比过滤
    group[group < group.quantile(.05)] = group.quantile(.05)
    group[group > group.quantile(.95)] = group.quantile(.95)
    return group

In [18]:
#对每条道路(link_ID)，每天执行(date)
df['travel_time'] = df.groupby(['link_ID','date'])['travel_time'].transform(quantile_clip)
df.head(5)

,link_ID,date,travel_time,time_interval_begin
0,4377906283422600514,2017-05-06,1.386294,2017-05-06 11:04:00
1,3377906289434510514,2017-05-06,0.693147,2017-05-06 10:42:00
2,3377906285934510514,2017-05-06,3.589059,2017-05-06 11:56:00
3,3377906285934510514,2017-05-06,3.303217,2017-05-06 17:46:00
4,3377906287934510514,2017-05-06,2.251292,2017-05-06 10:52:00


In [21]:
#根据需求选择样本数据，比如预测高峰时刻，如早上6-8、中午下午13-18
df = df.loc[(df['time_interval_begin'].dt.hour.isin([6,7,8,13,14,15,16,17,18]))]

In [23]:
#保存处理结果
df.to_csv('raw_data.txt',header=True,index=None,sep=';',mode='w')

### 缺失值预处理

In [25]:
df = pd.read_csv('raw_data.txt',delimiter=';',parse_dates=['time_interval_begin'],dtype={'link_ID':object})
df.head()

,link_ID,date,travel_time,time_interval_begin
0,3377906285934510514,2017-05-06,3.303217,2017-05-06 17:46:00
1,3377906287934510514,2017-05-06,1.887070,2017-05-06 14:36:00
2,3377906287674510514,2017-05-06,1.931521,2017-05-06 06:30:00
3,3377906287886510514,2017-05-06,3.616309,2017-05-06 07:32:00
4,4377906283759500514,2017-05-06,2.140066,2017-05-06 13:24:00


如上第一行中，2017-05-06 17:46:00，那么是不是没有17:48、17:50，所以我们需要补充

In [26]:
link_df.head()

,link_ID,length,width,link_class
0,4377906289869500514,57,3,1
1,4377906284594800514,247,9,1
2,4377906289425800514,194,3,1
3,4377906284525800514,839,3,1
4,4377906284422600514,55,12,1


In [27]:
date_range = pd.date_range('2017-03-01 00:00:00','2017-07-31 23:58:00',freq='2min')
date_range[:5]

DatetimeIndex(['2017-03-01 00:00:00', '2017-03-01 00:02:00',
               '2017-03-01 00:04:00', '2017-03-01 00:06:00',
               '2017-03-01 00:08:00'],
              dtype='datetime64[ns]', freq='2T')

In [29]:
#给每个ID，做出每一个时刻
new_index = pd.MultiIndex.from_product([link_df['link_ID'].unique(),date_range],
                                      names=['link_ID', 'time_interval_begin'])
new_df = pd.DataFrame(index=new_index).reset_index()
new_df.head()  # 此时每个ID都有从2017-03-01 00:00:00到2017-03-71 23:58:00的时间间隔

,link_ID,time_interval_begin
0,4377906289869500514,2017-03-01 00:00:00
1,4377906289869500514,2017-03-01 00:02:00
2,4377906289869500514,2017-03-01 00:04:00
3,4377906289869500514,2017-03-01 00:06:00
4,4377906289869500514,2017-03-01 00:08:00


In [32]:
#原来的表里也有部分值，进行合并，出现大量缺失值
df2 = pd.merge(new_df, df,on=['link_ID','time_interval_begin'],how='left')
df2.head()

,link_ID,time_interval_begin,date,travel_time
0,4377906289869500514,2017-03-01 00:00:00,NaN,NaN
1,4377906289869500514,2017-03-01 00:02:00,NaN,NaN
2,4377906289869500514,2017-03-01 00:04:00,NaN,NaN
3,4377906289869500514,2017-03-01 00:06:00,NaN,NaN
4,4377906289869500514,2017-03-01 00:08:00,NaN,NaN


In [41]:
#筛选时间段数据
df2 = df2.loc[(df2['time_interval_begin'].dt.hour.isin([6,7,8,13,14,15,16,17,18]))]
df2 = df2.loc[~((df2['time_interval_begin'].dt.year == 2017) & 
                (df2['time_interval_begin'].dt.month == 7) & 
                (df2['time_interval_begin'].dt.hour.isin([8,15,18])))]
df2 = df2.loc[~((df2['time_interval_begin'].dt.year == 2017) & 
                (df2['time_interval_begin'].dt.month == 3) & 
                (df2['time_interval_begin'].dt.day == 31))]

df2['date'] = df2['time_interval_begin'].dt.strftime('%Y-%m-%d')

In [42]:
df2.head()

,link_ID,time_interval_begin,date,travel_time
180,4377906289869500514,2017-03-01 06:00:00,2017-03-01,2.174752
181,4377906289869500514,2017-03-01 06:02:00,2017-03-01,2.174752
182,4377906289869500514,2017-03-01 06:04:00,2017-03-01,2.174752
183,4377906289869500514,2017-03-01 06:06:00,2017-03-01,2.174752
184,4377906289869500514,2017-03-01 06:08:00,2017-03-01,2.174752


In [ ]:
#保存中间结果
df2.to_csv('pre_trainning.txt',header=True,index=None,sep=';',mode='w')